In [ ]:
import pandas as pd
import pexpect
import re

def remove_ansi_codes(text):
    """
    ANSI escape 코드(스피너 등)를 제거합니다.
    """
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

def predict_category_pexpect(product_name, categories):
    """
    pexpect.run()을 사용하여 Ollama CLI를 호출한 후,
    예측된 카테고리를 추출합니다.
    """
    category_descriptions = {
        "패션 의류": "옷, 셔츠, 바지, 드레스, 재킷, 속옷 등 몸에 착용하는 의류 전반",
        "패션잡화": "가방, 신발, 모자, 벨트, 장갑, 목걸이, 팔찌 등 의류가 아닌 패션 액세서리",
        "화장품/미용": "스킨케어, 메이크업, 향수, 헤어케어 제품 등 피부 및 외모 관리 제품",
        "디지털/가전": "스마트폰, 노트북, TV, 카메라, 가전제품 등 전자기기 및 디지털 제품",
        "가구/인테리어": "침대, 소파, 테이블, 조명, 커튼 등 가구 및 실내 인테리어 관련 제품",
        "출산/육아": "유아용 의류, 기저귀, 유모차, 장난감 등 영유아 및 육아 관련 용품",
        "식품": "신선식품, 가공식품, 음료, 간식 등 먹을 수 있는 모든 식품 및 음료",
        "스포츠/레저": "운동복, 운동기구, 캠핑용품, 자전거 등 스포츠 및 야외 활동 관련 용품",
        "생활/건강": "청소용품, 생활용품, 건강보조식품, 위생용품, 샴푸, 바디워시 등 일상생활 필수품 및 건강 관련 제품",
        "여가/생활편의": "도서, 악기, 취미용품, 여행용품 등 여가 활동 및 자기 계발 관련 제품"
    }
    
    category_list = "\n".join([f"- {cat}: {desc}" for cat, desc in category_descriptions.items()])
    prompt = (
        "당신은 상품명을 기반으로 카테고리를 분류하는 역할을 합니다.\n"
        "주어진 상품명이 가장 적합한 단 하나의 카테고리를 선택해야 합니다.\n"
        "다른 말은 하지 말고, 반드시 가능한 카테고리 중 하나만 출력하세요.\n"
        "가능한 카테고리와 설명:\n"
        f"{category_list}\n"
        f"상품명: {product_name}"
    )
    
    cmd_str = f'ollama run EEVE-Korean-10.8B "{prompt}"'
    
    try:
        output = pexpect.run(cmd_str, encoding="utf-8", timeout=60)
        clean_output = remove_ansi_codes(output).strip()
        print(f"상품명: {product_name}\n예측된 카테고리 출력: {clean_output}\n")
        
        # 예측된 카테고리가 목록 중 하나인지 확인
        for category in category_descriptions.keys():
            if category in clean_output:
                return category
        
        print("예측된 카테고리가 목록에 없습니다. 기본값 설정.")
        return "미분류"
    except Exception as e:
        print(f"Ollama CLI 호출 중 오류 발생: {e}")
        return "미분류"

def main():
    # 예측 가능한 카테고리 목록
    categories = [
        "패션 의류", "패션잡화", "화장품/미용",
        "디지털/가전", "가구/인테리어", "출산/육아",
        "식품", "스포츠/레저", "생활/건강", "여가/생활편의"
    ]
    
    # CSV 파일 이름에서 원래 카테고리 추출 (예: "패션 의류.csv" -> "패션 의류")
    file_name = "패션 의류.csv"
    original_category = file_name.split('.')[0]
    
    # CSV 파일 읽기 및 '카테고리' 열 추가
    df = pd.read_csv(file_name)
    df['원래 카테고리'] = original_category
    df['예측 카테고리'] = df['Product Name'].apply(lambda x: predict_category_pexpect(x, categories))
    
    # 예측 결과를 바탕으로 데이터 분류
    merge_rows = df[df['원래 카테고리'] == df['예측 카테고리']]
    review_rows = df[df['원래 카테고리'] != df['예측 카테고리']]
    
    # 결과 CSV 저장
    if not merge_rows.empty:
        merge_rows.to_csv("merge.csv", index=False)
        print("검증 통과한 데이터는 merge.csv에 저장되었습니다.")
    else:
        print("검증 통과한 데이터가 없습니다.")
    
    if not review_rows.empty:
        review_rows.to_csv("review.csv", index=False)
        print("검증 실패한 데이터는 review.csv에 저장되었습니다.")
    else:
        print("검증 실패한 데이터가 없습니다.")

if __name__ == "__main__":
    main()